In [1]:
import os,glob
from PIL import Image
import numpy as np
from keras.utils import to_categorical
from keras.models import load_model

import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

Using TensorFlow backend.


In [ ]:
img_size = 56
work_dir = 'archive/lfw-deepfunneled/Smile_NotSmile'
image_dir = work_dir+"/original/unknown"
pred_dir = work_dir+"/predicted"
results_base='Smile_NotSmile-results'
model_file="Smile_NotSmile-model-size{0}.hd5".format(img_size)
classes=["Smile","NotSmile"]
num_classes=len(classes)

In [ ]:
model=load_model(model_file)

In [ ]:
#結果保存用ファイルを開く
results_file = '{0}-size{1}.csv'.format(results_base,img_size)
res = open(results_file,'w')
res.write("file,{0},{1}\n".format(classes[0],classes[1]))

#出力用ディレクトリを空にする
files=glob.glob(pred_dir+"/*.*")
for f in files:
    os.remove(f)

#ファイルひとつずつに対して予測
files=glob.glob(image_dir+"/*.*")
for f in files:
    #画像データの読み込み
    img=Image.open(f)
    img=img.convert("RGB")
    img=img.resize((img_size,img_size))
    #4次元のnumpy配列にreshape
    X=np.array(img).reshape(1,img_size,img_size,3).astype("float16")
    X_pred=X/255
    #分類予測
    pred=model.predict(X_pred)
    #画像をpred_dirに出力
    pred_ans=pred.argmax()
    pred_cls=classes[pred_ans]
    tag="pred_as_{0}-size{1}".format(pred_cls,img_size)
    base=os.path.basename(f)
    img_file="{0}/{1}.{2}.jpg".format(pred_dir,os.path.splitext(base)[0],tag)
    img.save(img_file)
    print(img_file,pred)
    res.write("{0},{1},{2}\n".format(f,pred[0][0],pred[0][1]))

#結果保存用ファイルを閉じる
res.close()

print("OK")